# Matrix Question Type Implementation for Odoo Quiz Module

This notebook documents the implementation of a Matrix Question type in the Quiz Engine Pro module. Matrix questions present users with a grid where they select cells based on the question prompt (similar to "mark all that apply" but in a table format).

## Overview of Matrix Questions

Matrix questions display a grid where:
- Rows typically represent statements or items
- Columns typically represent categories or options
- Users select cells to indicate relationships between rows and columns

## Model Structure

We've implemented three main models to support matrix questions:

1. `quiz.matrix.row` - Defines the rows of the matrix
2. `quiz.matrix.column` - Defines the columns of the matrix
3. `quiz.matrix.cell` - Defines the cells and their correct values

We also extended the existing `quiz.question` model to support the matrix question type.

In [ ]:
# Sample code for the matrix question models

# Matrix Row Model
"""
class MatrixRow(models.Model):
    _name = 'quiz.matrix.row'
    _description = 'Matrix Question Row'
    _order = 'sequence, id'
    
    sequence = fields.Integer(string='Sequence', default=10)
    question_id = fields.Many2one('quiz.question', string='Question', required=True, ondelete='cascade')
    name = fields.Char(string='Row Label', required=True)
    description = fields.Text(string='Description', help='Optional description or context for this row')
"""

# Matrix Column Model
"""
class MatrixColumn(models.Model):
    _name = 'quiz.matrix.column'
    _description = 'Matrix Question Column'
    _order = 'sequence, id'
    
    sequence = fields.Integer(string='Sequence', default=10)
    question_id = fields.Many2one('quiz.question', string='Question', required=True, ondelete='cascade')
    name = fields.Char(string='Column Label', required=True)
    description = fields.Text(string='Description', help='Optional description or context for this column')
"""

# Matrix Cell Model
"""
class MatrixCell(models.Model):
    _name = 'quiz.matrix.cell'
    _description = 'Matrix Question Cell'
    
    question_id = fields.Many2one('quiz.question', string='Question', related='row_id.question_id', store=True)
    row_id = fields.Many2one('quiz.matrix.row', string='Row', required=True, ondelete='cascade')
    column_id = fields.Many2one('quiz.matrix.column', string='Column', required=True, ondelete='cascade')
    is_correct = fields.Boolean(string='Is Correct', default=False,
                               help='Mark as correct if this cell should be selected for the correct answer')
"""

## Extension to the Question Model

We extended the `quiz.question` model to support matrix questions by:

1. Adding a new question type 'matrix' to the selection field
2. Adding One2many fields to connect to the matrix models
3. Updating the constraint methods to validate matrix questions

In [ ]:
# Extension to the Question Model

"""
# Add to the type selection
type = fields.Selection([
    # ... existing types ...
    ('matrix', 'Matrix Question')
], string='Type', default='mcq_single', required=True)

# Add matrix relationships
matrix_row_ids = fields.One2many('quiz.matrix.row', 'question_id', string='Matrix Rows')
matrix_column_ids = fields.One2many('quiz.matrix.column', 'question_id', string='Matrix Columns')
matrix_cell_ids = fields.One2many('quiz.matrix.cell', 'question_id', string='Matrix Cells')

# Update constraints
@api.constrains('type')
def _check_required_fields(self):
    for question in self:
        # ... existing constraints ...
        elif question.type == 'matrix':
            if not question.matrix_row_ids:
                raise ValidationError(_('Matrix questions must have rows defined.'))
            if not question.matrix_column_ids:
                raise ValidationError(_('Matrix questions must have columns defined.'))
"""

## Views Implementation

We created several views to support matrix questions:

1. Form and tree views for matrix rows, columns, and cells
2. Extension to the question form view to add a "Matrix Configuration" tab
3. Website template for displaying matrix questions to users

In [ ]:
# XML View for Matrix Configuration in Question Form

"""
<record id="view_question_form_with_matrix" model="ir.ui.view">
    <field name="name">quiz.question.form.matrix</field>
    <field name="model">quiz.question</field>
    <field name="inherit_id" ref="quiz_engine_pro.view_quiz_question_form"/>
    <field name="arch" type="xml">
        <notebook position="inside">
            <page string="Matrix Configuration" attrs="{'invisible': [('type', '!=', 'matrix')]}">
                <div class="row">
                    <div class="col-6">
                        <h3>Matrix Rows</h3>
                        <field name="matrix_row_ids" context="{'default_question_id': active_id}">
                            <tree editable="bottom">
                                <field name="sequence" widget="handle"/>
                                <field name="name"/>
                                <field name="question_id" invisible="1"/>
                            </tree>
                        </field>
                    </div>
                    <div class="col-6">
                        <h3>Matrix Columns</h3>
                        <field name="matrix_column_ids" context="{'default_question_id': active_id}">
                            <tree editable="bottom">
                                <field name="sequence" widget="handle"/>
                                <field name="name"/>
                                <field name="question_id" invisible="1"/>
                            </tree>
                        </field>
                    </div>
                </div>
                <div class="mt-3">
                    <h3>Matrix Cell Configuration</h3>
                    <p class="text-muted">Check the cells that represent the correct answers for your matrix question.</p>
                    <field name="matrix_cell_ids" context="{'default_question_id': active_id}">
                        <tree editable="bottom">
                            <field name="row_id" domain="[('question_id', '=', parent.id)]"/>
                            <field name="column_id" domain="[('question_id', '=', parent.id)]"/>
                            <field name="is_correct"/>
                            <field name="question_id" invisible="1"/>
                        </tree>
                    </field>
                </div>
            </page>
        </notebook>
    </field>
</record>
"""

## Frontend Template

The frontend template displays the matrix as a table with checkboxes. Users can select cells, and their selections are stored as JSON data for evaluation.

In [ ]:
# Frontend Template for Matrix Questions

"""
<t t-if="question.type == 'matrix'">
    <div class="matrix-question">
        <input type="hidden" name="matrix_data" value="{}"/>
        
        <div class="table-responsive">
            <table class="table table-bordered">
                <thead>
                    <tr>
                        <th></th> <!-- Empty corner cell -->
                        <t t-foreach="question.matrix_column_ids" t-as="column">
                            <th class="text-center"><t t-esc="column.name"/></th>
                        </t>
                    </tr>
                </thead>
                <tbody>
                    <t t-foreach="question.matrix_row_ids" t-as="row">
                        <tr>
                            <th><t t-esc="row.name"/></th>
                            <t t-foreach="question.matrix_column_ids" t-as="column">
                                <td class="text-center">
                                    <div class="form-check d-flex justify-content-center">
                                        <input type="checkbox" 
                                               class="form-check-input matrix-cell" 
                                               t-att-data-row-id="str(row.id)"
                                               t-att-data-col-id="str(column.id)"
                                               t-att-id="'cell_' + str(row.id) + '_' + str(column.id)"/>
                                    </div>
                                </td>
                            </t>
                        </tr>
                    </t>
                </tbody>
            </table>
        </div>
        
        <script type="text/javascript">
            document.addEventListener('DOMContentLoaded', function() {
                // Update hidden form field when matrix cells are clicked
                var matrixCells = document.querySelectorAll('.matrix-cell');
                var hiddenField = document.querySelector('input[name="answer_data"]');
                
                if (!hiddenField) {
                    // Create the hidden field if it doesn't exist
                    hiddenField = document.createElement('input');
                    hiddenField.type = 'hidden';
                    hiddenField.name = 'answer_data';
                    document.querySelector('.matrix-question').appendChild(hiddenField);
                }
                
                var matrixData = {};
                
                matrixCells.forEach(function(cell) {
                    cell.addEventListener('change', function() {
                        var rowId = this.getAttribute('data-row-id');
                        var colId = this.getAttribute('data-col-id');
                        var cellKey = 'cell_' + rowId + '_' + colId;
                        
                        matrixData[cellKey] = this.checked;
                        hiddenField.value = JSON.stringify(matrixData);
                    });
                });
            });
        </script>
    </div>
</t>
"""

## Answer Evaluation

For evaluating matrix questions, we've implemented an evaluation method that compares the user's answers to the correct cell configurations:

In [ ]:
# Matrix Answer Evaluation Code

"""
def _evaluate_matrix(self, answer_data):
    """Evaluate matrix questions"""
    if not answer_data:
        return 0.0
    
    try:
        answers = json.loads(answer_data) if isinstance(answer_data, str) else answer_data
    except Exception:
        return 0.0
    
    total_cells = len(self.matrix_row_ids) * len(self.matrix_column_ids)
    if total_cells == 0:
        return 0.0
    
    correct_count = 0
    
    for row in self.matrix_row_ids:
        for col in self.matrix_column_ids:
            cell_key = f"cell_{row.id}_{col.id}"
            expected_value = self._get_matrix_correct_value(row, col)
            
            if cell_key in answers and answers[cell_key] == expected_value:
                correct_count += 1
    
    return (correct_count / total_cells) * self.points

def _get_matrix_correct_value(self, row, col):
    """Get the correct value for a matrix cell"""
    cell = self.env['quiz.matrix.cell'].search([
        ('row_id', '=', row.id),
        ('column_id', '=', col.id)
    ], limit=1)
    
    return cell.is_correct if cell else False
"""

## Security Access Rights

We've configured access rights for all user types:

1. Regular users - Full CRUD access
2. Quiz Masters - Full CRUD access (special group for quiz management)
3. Public users - Read-only access

In [ ]:
# Security Access Rights Configuration

"""
# Access rights for internal users
access_quiz_matrix_row_user,quiz.matrix.row user,model_quiz_matrix_row,base.group_user,1,1,1,1
access_quiz_matrix_column_user,quiz.matrix.column user,model_quiz_matrix_column,base.group_user,1,1,1,1
access_quiz_matrix_cell_user,quiz.matrix.cell user,model_quiz_matrix_cell,base.group_user,1,1,1,1

# Access rights for quiz masters
access_quiz_matrix_row_master,quiz.matrix.row master,model_quiz_matrix_row,quiz_engine_pro.group_quiz_master,1,1,1,1
access_quiz_matrix_column_master,quiz.matrix.column master,model_quiz_matrix_column,quiz_engine_pro.group_quiz_master,1,1,1,1
access_quiz_matrix_cell_master,quiz.matrix.cell master,model_quiz_matrix_cell,quiz_engine_pro.group_quiz_master,1,1,1,1

# Access rights for public users (website visitors)
access_quiz_matrix_row_public,quiz.matrix.row public,model_quiz_matrix_row,base.group_public,1,0,0,0
access_quiz_matrix_column_public,quiz.matrix.column public,model_quiz_matrix_column,base.group_public,1,0,0,0
access_quiz_matrix_cell_public,quiz.matrix.cell public,model_quiz_matrix_cell,base.group_public,1,0,0,0
"""

## CSS Styling

We've created custom CSS for matrix questions to improve the visual appearance:

In [ ]:
# CSS Styling for Matrix Questions

"""
/* Matrix question styling */
.matrix-question {
    margin-bottom: 2rem;
}

.matrix-question table {
    background-color: #fff;
    border-collapse: collapse;
}

.matrix-question th {
    background-color: #f8f9fa;
    font-weight: 600;
}

.matrix-question .form-check {
    margin: 0;
    padding: 0;
    height: 100%;
    display: flex;
    justify-content: center;
    align-items: center;
}

.matrix-question .form-check-input {
    margin: 0;
    position: relative;
    transform: scale(1.2);
}

.matrix-question td {
    vertical-align: middle;
    padding: 8px;
}

/* Responsive tweaks */
@media (max-width: 767px) {
    .matrix-question th, .matrix-question td {
        padding: 5px;
        font-size: 0.9rem;
    }
    
    .matrix-question .form-check-input {
        transform: scale(1);
    }
}
"""

## Common Error: Fixing the RPC_ERROR

When implementing the matrix question type, we encountered an RPC_ERROR related to the view inheritance. The error was:

```
ValueError: External ID not found in the system: quiz_engine_pro.view_question_form
```

This happened because the view ID was incorrect. The solution was to check the correct ID in the question_views.xml file and use that:

```xml
<field name="inherit_id" ref="quiz_engine_pro.view_quiz_question_form"/>
```

This issue highlights the importance of checking the actual view IDs in Odoo before creating view inheritance.

## Usage Example

Here's how to create a matrix question using the new functionality:

1. Create a new question and select "Matrix Question" as the type
2. Go to the "Matrix Configuration" tab
3. Add rows for your statements (e.g., "The Earth is round", "Humans need oxygen to live")
4. Add columns for your options (e.g., "True", "False")
5. Mark the correct cells (e.g., both statements are true, so you'd mark the "True" cell for both rows)
6. Save and publish your quiz

## Testing the Matrix Question Type

To test a matrix question:

1. Create a quiz with a matrix question as described above
2. Preview or publish the quiz
3. Select cells in the matrix based on the question prompt
4. Submit the answer
5. Verify that the evaluation logic correctly scores the answer

The system evaluates the matrix questions by comparing user selections with the defined correct cells. Partial credit is awarded based on the number of correct cells selected.

## Conclusion

The matrix question type enhances the Quiz Engine Pro module by adding a new, structured question format. This format is particularly useful for:

- Knowledge assessments where users need to categorize multiple items
- Surveys where users need to rate multiple aspects on a scale
- Tests where users need to match multiple items to categories

The implementation follows Odoo best practices for model structure, views, and security access controls.